In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install mido
import numpy as np
import tensorflow as tf
#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
from tensorflow.keras.layers import Input, LSTM, Dropout, Dense, Flatten, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Model

In [ ]:
#Run when you are required to check the new added midi files in the datasets

midy = []
to_delete = []


import glob
import mido
from mido import MidiFile, MidiTrack, Message
for midi in glob.glob('/content/drive/MyDrive/Major Project/datasets/adl-piano-midi/*.mid'):
  if midi not in midy:
    try:
      print(midi)
      mid = MidiFile(midi)
      midy.append(midi)
    except:
      to_delete.append(midi)


to_delete_files = "\n".join(to_delete)
with open("/content/drive/MyDrive/Major Project/output/to_delete.txt", "w+") as file:
  file.write(to_delete_files)


import os
with open("/content/drive/MyDrive/Major Project/output/to_delete.txt", "r+") as file:
  for line in file:
    try:
      os.remove(line.rstrip())
    except FileNotFoundError:
      print("There is no such file")

In [ ]:
import glob
import mido
from mido import MidiFile, MidiTrack, Message
#notes = []
#times = []
count = 0
for folder in glob.glob('/content/drive/MyDrive/Major Project/datasets/*'):
  for midi in glob.glob(folder + '/*.mid'):
    #print(midi)

    if count > 10416:
      print(count)
      try:
        mid = MidiFile(midi)
        for msg in mid:
            if not msg.is_meta and msg.type == 'note_on':
              notes.append(msg.note)
              times.append(msg.time)
              
      except:
        to_delete.append(midi)
        print("error " + str(count))

    count += 1

In [2]:
notes = np.load("/content/drive/MyDrive/Major Project/datasets/maestro_notes.npy")
times = np.load("/content/drive/MyDrive/Major Project/datasets/maestro_times.npy")
notes = list(notes)
times = list(times)

In [4]:
len(notes)

14080328

In [3]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit(np.array(notes).reshape(-1, 1))
scaled_notes = list(scaler.transform(np.array(notes).reshape(-1, 1)))

In [4]:
scaled_times = [max(0, -np.log10(time + 0.0001)) for time in times]
scaled_times = list(np.array(scaled_times).reshape(-1, 1))
max(scaled_times)

array([4.])

In [ ]:
final_notes = [10**(-time) for time in tym]
min(final_notes)

In [7]:
scaled_times[100:110]

[array([1.48956671]),
 array([2.66087995]),
 array([1.25720943]),
 array([2.36991129]),
 array([2.13125763]),
 array([1.32813321]),
 array([2.02342078]),
 array([1.58266673]),
 array([2.27504181]),
 array([1.11837762])]

In [ ]:
pitchnames = sorted(set(item for item in notes))
print(pitchnames)
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
print(note_to_int)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 14: 13, 16: 14, 17: 15, 18: 16, 19: 17, 20: 18, 21: 19, 22: 20, 23: 21, 24: 22, 25: 23, 26: 24, 27: 25, 28: 26, 29: 27, 30: 28, 31: 29, 32: 30, 33: 31, 34: 32, 35: 33, 36: 34, 37: 35, 38: 36, 39: 37, 40: 38, 41: 39, 42: 40, 43: 41, 44: 42, 45: 43, 46: 44, 47: 45, 48: 46, 49: 47, 50: 48, 51: 49, 52: 50, 53: 51, 54: 52, 55: 53, 56: 54, 57: 55, 58: 56, 59: 57, 60: 58, 61: 59, 62: 60, 63: 61, 

In [5]:
notes_with_times = []
i = 0
for note in list(scaled_notes):
    temp = []
    temp.append(note)
    temp.append(list(scaled_times[i]))
    notes_with_times.append(temp)
    i += 1
notes_with_times = np.array(notes_with_times).reshape(-1, 2)
notes_with_times.shape

(14080328, 2)

In [6]:
notes_with_times = [list(note) for note in notes_with_times]

X = []

# number of notes in a sequence
n_prev = 100
#for i in range(0, len(notes_with_times)-n_prev, 10):
  #X.append(notes_with_times[i : i + n_prev])
for i in range(0, len(notes_with_times) - n_prev, n_prev):
  X.append(notes_with_times[i : i + n_prev])

In [7]:
X = np.array(X)
print(X[0,22,:])
print(X.shape)
#np.random.shuffle(X)
#print(X[0,22,:])
#print(X.shape)

[0.34482759 1.29122512]
(140803, 100, 2)


In [8]:
del notes
del times
del notes_with_times

In [15]:
def generator():
    gen_input = Input(shape = (None, 100))
    X = LSTM(512, return_sequences = True)(gen_input)
    X = BatchNormalization(momentum = 0.8)(X)
    X = LeakyReLU(alpha = 0.2)(X)
    X = LSTM(256, return_sequences = True)(X)
    X = BatchNormalization(momentum = 0.8)(X)
    X = LeakyReLU(alpha = 0.2)(X)
    X = Dense(128)(X)
    X = LeakyReLU(alpha = 0.2)(X)
    X = BatchNormalization(momentum = 0.8)(X)
    X = Dropout(0.5)(X)
    out = Dense(2, activation = "tanh")(X)
    model = Model(inputs = gen_input, outputs = out)
    return model

In [16]:
def discriminator():
    disc_input = Input(shape = (100, 2))
    X = LSTM(512, return_sequences = True)(disc_input)
    X = LeakyReLU(alpha = 0.2)(X)
    X = LSTM(256, return_sequences = True)(X)
    X = LeakyReLU(alpha = 0.2)(X)
    X = Flatten()(X)
    X = Dense(128)(X)
    X = LeakyReLU(alpha = 0.2)(X)
    out = Dense(1, activation = "sigmoid")(X)
    model = Model(inputs = disc_input, outputs = out)
    return model

In [17]:
def disc_with_gen(generator, discriminator):
    inputs = Input(shape = (None, 100))
    generated = generator(inputs)
    outputs = discriminator(generated)
    model = Model(inputs = inputs, outputs = outputs)
    return model

In [18]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(0.0002, 0.5)
d = discriminator()
d.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
g = generator()

In [19]:
d.trainable = False
combined = disc_with_gen(g, d)
combined.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [20]:
combined.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None, 100)]       0         
_________________________________________________________________
model_1 (Functional)         (None, None, 2)           2079618   
_________________________________________________________________
model (Functional)           (None, 1)                 5119233   
Total params: 7,198,851
Trainable params: 2,077,826
Non-trainable params: 5,121,025
_________________________________________________________________


In [21]:
from matplotlib import pyplot as plt
assert not np.any(np.isnan(X))

In [22]:
disc_loss = []
disc_loss_epoch = []
gen_loss = []
gen_loss_epoch = []
batch_size = 128

In [23]:
idx = list(range(0, batch_size, 1))
real_data = X[idx]
real_data.shape

(128, 100, 2)

In [24]:
def train(epochs, num_of_batches, batch_size = batch_size):
    for epoch in range(epochs):
        a = 0
        b = batch_size
        for num in range(num_of_batches):
          idx = list(range(a, b, 1))
          real_data = X[idx]

          noise = np.random.normal(0, 1, (batch_size, 100, 100))
        
          generated = g.predict(noise)
          d_loss_real = d.train_on_batch(real_data, np.ones((batch_size, 1)))
          #print("d_loss_real: ", d_loss_real)
          d_loss_fake = d.train_on_batch(generated, np.zeros((batch_size, 1)))
          #print("d_loss_fake: ",d_loss_fake)
          d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
          print("%d d_loss: " %epoch,d_loss)

          for _ in range(5):            
            noise = np.random.normal(0, 1, (batch_size, 100, 100))
            g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))
          print("%d g_loss: " %epoch, g_loss)

          print("\n")

          disc_loss.append(d_loss[0])
          gen_loss.append(g_loss[0])

          a = a + batch_size
          b = b + batch_size
            
        if epoch % 1 == 0:
            print ("Epoch %d [D loss: %f, acc.: %.2f%%] [G loss: %f, acc.: %.2f%%]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0], 100*g_loss[1]))
            print("\n")
            disc_loss_epoch.append(d_loss[0])
            gen_loss_epoch.append(g_loss[0])
            save_model()
            save_weights()
            save_losses()
            checkpoint = tf.train.Checkpoint(g)
            checkpoint.save('/content/drive/MyDrive/Major Project/arch_bc/checkpoints/training_checkpoints')
            
    plt.plot(disc_loss, c='red')
    plt.plot(gen_loss, c='blue')
    plt.title("GAN Loss per batch")
    plt.legend(['Discriminator', 'Generator'])
    plt.xlabel('Batch')
    plt.ylabel('Loss')
    plt.ylim([0, 10])
    plt.savefig('/content/drive/MyDrive/Major Project/arch_bc/GAN_Loss_per_Batch_final.png', transparent=False)
    plt.close()

    plt.plot(disc_loss_epoch, c='red')
    plt.plot(gen_loss_epoch, c='blue')
    plt.title("GAN Loss per Epoch")
    plt.legend(['Discriminator', 'Generator'])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.savefig('/content/drive/MyDrive/Major Project/arch_bc/GAN_Loss_per_Epoch_final.png', transparent=False)
    plt.close()

In [25]:
print(real_data.shape)
generated = g.predict(np.random.normal(0, 1, (batch_size, 100, 100)))
print(generated.shape)
#idx = list(range(0, batch_size, 1))
#print(real_data[idx])

(128, 100, 2)
(128, 100, 2)


In [ ]:
train(epochs = 20, num_of_batches = int(X.shape[0]/batch_size))

Streaming output truncated to the last 5000 lines.

9 d_loss:  [7.75695086e-06 1.00000000e+00]
9 g_loss:  [11.07744026184082, 0.0]


9 d_loss:  [7.74159344e-06 1.00000000e+00]
9 g_loss:  [11.0791015625, 0.0]


9 d_loss:  [7.72882049e-06 1.00000000e+00]
9 g_loss:  [11.08098030090332, 0.0]


9 d_loss:  [7.71281619e-06 1.00000000e+00]
9 g_loss:  [11.083161354064941, 0.0]


9 d_loss:  [7.79770028e-06 1.00000000e+00]
9 g_loss:  [11.084922790527344, 0.0]


9 d_loss:  [7.6849883e-06 1.0000000e+00]
9 g_loss:  [11.086616516113281, 0.0]


9 d_loss:  [7.68003215e-06 1.00000000e+00]
9 g_loss:  [11.088509559631348, 0.0]


9 d_loss:  [2.01723042e-05 1.00000000e+00]
9 g_loss:  [11.086130142211914, 0.0]


9 d_loss:  [7.68209793e-06 1.00000000e+00]
9 g_loss:  [11.0869779586792, 0.0]


9 d_loss:  [7.66602969e-06 1.00000000e+00]
9 g_loss:  [11.08879280090332, 0.0]


9 d_loss:  [7.65694949e-06 1.00000000e+00]
9 g_loss:  [11.09000015258789, 0.0]


9 d_loss:  [7.64574088e-06 1.00000000e+00]
9 g_loss:  [11.0

In [9]:
def save_weights():
  g.save_weights("/content/drive/MyDrive/Major Project/arch_bc/weights/gen_model_weights_time_maestro.h5")
  d.save_weights("/content/drive/MyDrive/Major Project/arch_bc/weights/disc_model_weights_time_maestro.h5")
  combined.save_weights("/content/drive/MyDrive/Major Project/arch_bc/weights/combined_model_weights_time_maestro.h5")

In [ ]:
#g.load_weights("/content/drive/MyDrive/Major Project/saved_weights/Model_weights_time_128_100_1_500k_data_repeating_50k.h5")

In [10]:
def save_model():
  d.trainable = False
  combined.save("/content/drive/MyDrive/Major Project/arch_bc/models/combined_model_time_maestro.h5")
  d.trainable = True
  g.save("/content/drive/MyDrive/Major Project/arch_bc/models/gen_model_time_maestro.h5")
  d.save("/content/drive/MyDrive/Major Project/arch_bc/models/disc_model_time_maestro.h5")

In [ ]:
g = tf.keras.models.load_model("/content/drive/MyDrive/Major Project/arch_bc/models/gen_model_time_maestro.h5")
d = tf.keras.models.load_model("/content/drive/MyDrive/Major Project/arch_bc/models/disc_model_time_maestro.h5")
combined = tf.keras.models.load_model("/content/drive/MyDrive/Major Project/arch_bc/models/combined_model_time_maestro.h5")

In [11]:
def save_losses():
  np.save("/content/drive/MyDrive/Major Project/arch_bc/losses/disc_loss.npy", np.array(disc_loss))
  np.save("/content/drive/MyDrive/Major Project/arch_bc/losses/disc_loss_epoch.npy", np.array(disc_loss_epoch))
  np.save("/content/drive/MyDrive/Major Project/arch_bc/losses/gen_loss.npy", np.array(gen_loss))
  np.save("/content/drive/MyDrive/Major Project/arch_bc/losses/gen_loss_epoch.npy", np.array(gen_loss_epoch))

In [13]:
disc_loss = list(np.load("/content/drive/MyDrive/Major Project/arch_bc/losses/disc_loss.npy"))
disc_loss_epoch = list(np.load("/content/drive/MyDrive/Major Project/arch_bc/losses/disc_loss_epoch.npy"))
gen_loss = list(np.load("/content/drive/MyDrive/Major Project/arch_bc/losses/gen_loss.npy"))
gen_loss_epoch = list(np.load("/content/drive/MyDrive/Major Project/arch_bc/losses/gen_loss_epoch.npy"))

In [ ]:
pred = g.predict(np.random.normal(0, 1, (1, 100, 100)))
d.predict(pred)

array([[0.39502835]], dtype=float32)

In [ ]:
d.predict(X[100, :, :].reshape(1, 100, 2))

array([[0.37082332]], dtype=float32)

In [ ]:
prediction = g.predict(np.random.normal(0, 1, (1, 100, 100)))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]
print(prediction)
#prediction = np.argmax(prediction, -1)

In [ ]:
prediction = g.predict(np.random.normal(0, 1, (1, 100, 100)))
print(prediction)

[[[0.5778004  0.10240071]
  [0.5495404  0.10249842]
  [0.5469154  0.10253565]
  [0.5416372  0.10209853]
  [0.5575677  0.10488557]
  [0.57445395 0.10791881]
  [0.558133   0.10502017]
  [0.5378927  0.10170298]
  [0.5385946  0.10177713]
  [0.5401536  0.10194181]
  [0.5397788  0.10190222]
  [0.61130476 0.10081137]
  [0.5384033  0.10175692]
  [0.5369209  0.10160033]
  [0.5399188  0.101917  ]
  [0.5393499  0.10185692]
  [0.5379783  0.10171202]
  [0.5363787  0.10154305]
  [0.5368197  0.10158964]
  [0.53859264 0.10177692]
  [0.5764251  0.10132926]
  [0.53947693 0.10187033]
  [0.5409608  0.10202708]
  [0.53812563 0.10172759]
  [0.540082   0.10193425]
  [0.54059416 0.10198835]
  [0.5425387  0.10219376]
  [0.54559773 0.10185206]
  [0.54092515 0.10202331]
  [0.54178095 0.10211372]
  [0.5391195  0.10183258]
  [0.53897893 0.10181773]
  [0.53813446 0.10172852]
  [0.5385645  0.10177395]
  [0.538113   0.10172626]
  [0.5379492  0.10170895]
  [0.53793395 0.10170734]
  [0.5397756  0.10190188]
  [0.5382917

In [ ]:
prediction = g.predict(np.random.normal(0, 1, (2, 100, 100)))
predicted_notes = []
predicted_times = []
for p in prediction.reshape(-1, 2):
    predicted_notes.append(p[0])
    predicted_times.append(p[1])
print(np.array(predicted_notes).shape)
print(np.array(predicted_times).shape)

(200,)
(200,)


In [ ]:
min(predicted_times)

0.083920315

In [ ]:
notes_scaler = MinMaxScaler(feature_range = (0, 1))
notes_scaler.fit(np.array(notes).reshape(-1, 1))
_notes = np.squeeze(notes_scaler.inverse_transform(np.array(predicted_notes).reshape(-1,1)))
final_notes = [int(i) for i in _notes]
print(final_notes)

[71, 72, 68, 72, 72, 68, 68, 68, 69, 112, 68, 68, 68, 68, 68, 68, 68, 68, 69, 68, 69, 68, 68, 68, 68, 68, 68, 68, 69, 68, 68, 68, 69, 84, 69, 68, 68, 68, 68, 68, 69, 68, 68, 68, 68, 68, 68, 68, 68, 73, 68, 68, 68, 68, 68, 68, 68, 68, 68, 76, 71, 68, 69, 68, 68, 68, 68, 69, 69, 69, 68, 68, 68, 69, 68, 69, 79, 68, 68, 82, 70, 69, 69, 68, 68, 68, 68, 68, 68, 69, 76, 68, 68, 68, 68, 68, 68, 68, 68, 68, 73, 71, 71, 68, 71, 68, 68, 68, 68, 67, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 69, 68, 68, 68, 68, 68, 68, 68, 68, 69, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 83, 68, 69, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 66, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 66, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 66, 68, 68, 68, 68, 68, 72, 68]


In [ ]:
scaler = MinMaxScaler(feature_range = (0.1, 1))
scaler.fit(np.array(times).reshape(-1, 1))
final_times = np.squeeze(scaler.inverse_transform(np.array(predicted_times).reshape(-1,1)))
for i in range(200):
    if final_times[i] < 0:
        final_times[i] = 0
print(final_times)

[0.         0.30915308 1.6390212  1.3250817  0.9340133  0.9036184
 0.06144243 0.         0.         0.77979654 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.2650493  0.         0.
 0.         0.02836622 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.64162964
 0.         0.608827   0.8189893  0.         0.36084917 0.03911702
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.05801363 0.09226353
 0.         0.00884301 0.         0.         0.         0.
 0.         0.         0.23654379 0.         0.01538574 0.
 0.04446698 0.         0.         0.02791138 0.14857484 0.
 0.1827643  0.0184583  0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         1.3386347  0.16880417 0.         0.
 0.2088716  0.519185   0.86392    0.08777872 0.         0.
 0.         0.         0.

In [ ]:
from mido import MidiFile, MidiTrack, Message
mid = MidiFile()
track = MidiTrack()
for i in range(200):
    # 147 means note_on
    # 67 is velocity
    if final_notes[i] > 127:
      final_notes[i] = 127
    note = np.asarray([147, final_notes[i], 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    time = final_times[i]
    msg.time = round(time * mid.ticks_per_beat * 2)
    track.append(msg)
mid.tracks.append(track)
mid.save('/content/drive/MyDrive/Major Project/output/LSTM_music_time_check66.mid')

In [ ]:
from mido import MidiFile, MidiTrack, Message
mid = MidiFile()
track = MidiTrack()
t = 230
track.append(Message('program_change', program = 0, channel = 0, time = 0))
for note in final_notes:
    # 147 means note_on
    # 67 is velocity
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    if t >= 160:
        t -= 2
    msg.time = int(t)
    msg.channel = 0
    track.append(msg)
mid.tracks.append(track)
mid.save('/content/drive/MyDrive/Major Project/output/LSTM_music.mid')
for msg in mid:
  print(msg)

program_change channel=0 program=0 time=0
note_on channel=0 note=59 velocity=67 time=0.2375
note_on channel=0 note=58 velocity=67 time=0.23541666666666666
note_on channel=0 note=56 velocity=67 time=0.23333333333333334
note_on channel=0 note=57 velocity=67 time=0.23124999999999998
note_on channel=0 note=57 velocity=67 time=0.22916666666666666
note_on channel=0 note=57 velocity=67 time=0.22708333333333333
note_on channel=0 note=58 velocity=67 time=0.225
note_on channel=0 note=59 velocity=67 time=0.22291666666666665
note_on channel=0 note=59 velocity=67 time=0.22083333333333333
note_on channel=0 note=65 velocity=67 time=0.21875
note_on channel=0 note=59 velocity=67 time=0.21666666666666667
note_on channel=0 note=59 velocity=67 time=0.21458333333333332
note_on channel=0 note=59 velocity=67 time=0.2125
note_on channel=0 note=59 velocity=67 time=0.21041666666666667
note_on channel=0 note=59 velocity=67 time=0.20833333333333334
note_on channel=0 note=59 velocity=67 time=0.20625
note_on channe

In [ ]:
import mido
from mido import MidiFile, MidiTrack, Message
import numpy as np

mid = MidiFile('/content/drive/MyDrive/Major Project/output/LSTM_music.mid')
#mid = MidiFile('pnowks/corner.mid') 
notes = []

for msg in mid:
    print(msg)

note_on channel=3 note=69 velocity=67 time=0.2375
note_on channel=3 note=46 velocity=67 time=0.23541666666666666
note_on channel=3 note=89 velocity=67 time=0.23333333333333334
note_on channel=3 note=57 velocity=67 time=0.23124999999999998
note_on channel=3 note=58 velocity=67 time=0.22916666666666666
note_on channel=3 note=64 velocity=67 time=0.22708333333333333
note_on channel=3 note=67 velocity=67 time=0.225
note_on channel=3 note=59 velocity=67 time=0.22291666666666665
note_on channel=3 note=53 velocity=67 time=0.22083333333333333
note_on channel=3 note=53 velocity=67 time=0.21875
note_on channel=3 note=58 velocity=67 time=0.21666666666666667
note_on channel=3 note=56 velocity=67 time=0.21458333333333332
note_on channel=3 note=74 velocity=67 time=0.2125
note_on channel=3 note=69 velocity=67 time=0.21041666666666667
note_on channel=3 note=52 velocity=67 time=0.20833333333333334
note_on channel=3 note=59 velocity=67 time=0.20625
note_on channel=3 note=48 velocity=67 time=0.20416666666

In [ ]:
b = [1,2,3,4,5,6,7,8,9]
a = list(np.array(b).reshape(-1,1))
note = [list(v) for v in a]
print(a)

[array([1]), array([2]), array([3]), array([4]), array([5]), array([6]), array([7]), array([8]), array([9])]


In [ ]:
import mido
from mido import MidiFile, MidiTrack, Message
import numpy as np

#mid = MidiFile('/content/drive/MyDrive/Major Project/output/LSTM_music.mid')
mid = MidiFile('corner.mid')
midy = MidiFile()
track = MidiTrack()

start = 0

midy.ticks_per_beat = mid.ticks_per_beat

for msg in mid:
    msg.time = round(msg.time * midy.ticks_per_beat * 2)
    if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
        msg.note = prediction[start]
        start += 1
        track.append(msg)
        if start >= len(prediction):
            break
    else:
        track.append(msg)
midy.tracks.append(track)
midy.save("/content/drive/MyDrive/Major Project/output/lstmaa.mid")

In [ ]:
for msg in mid:
    print(msg)

<meta message track_name name='untitled' time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message key_signature key='C' time=0>
<meta message set_tempo tempo=500000 time=0>
<meta message midi_port port=0 time=0>
<meta message track_name name='Piano' time=0>
control_change channel=0 control=0 value=0 time=0
control_change channel=0 control=32 value=0 time=0
program_change channel=0 program=0 time=0
control_change channel=0 control=64 value=127 time=2.0708333333333333
control_change channel=0 control=64 value=0 time=0.10833333333333334
control_change channel=0 control=64 value=127 time=0.2833333333333333
control_change channel=0 control=67 value=127 time=0.0375
note_on channel=0 note=36 velocity=54 time=0.13333333333333333
note_on channel=0 note=48 velocity=73 time=0.25416666666666665
note_on channel=0 note=55 velocity=52 time=0.5125
note_on channel=0 note=60 velocity=60 time=0.12083333333333333
note_on channel

note_on channel=0 note=54 velocity=0 time=0.008333333333333333
note_on channel=0 note=63 velocity=0 time=0.020833333333333332
note_on channel=0 note=46 velocity=0 time=0.09166666666666666
note_on channel=0 note=63 velocity=73 time=0.03333333333333333
note_on channel=0 note=54 velocity=62 time=0.0125
note_on channel=0 note=58 velocity=62 time=0.008333333333333333
note_on channel=0 note=54 velocity=0 time=0.025
note_on channel=0 note=58 velocity=0 time=0.016666666666666666
note_on channel=0 note=63 velocity=0 time=0.020833333333333332
control_change channel=0 control=64 value=0 time=0.17916666666666667
note_on channel=0 note=44 velocity=64 time=0.0125
note_on channel=0 note=61 velocity=82 time=0.004166666666666667
note_on channel=0 note=56 velocity=70 time=0.004166666666666667
note_on channel=0 note=53 velocity=69 time=0.016666666666666666
control_change channel=0 control=64 value=127 time=0
note_on channel=0 note=53 velocity=0 time=0.075
note_on channel=0 note=56 velocity=0 time=0.00416

note_on channel=0 note=80 velocity=103 time=0.0375
note_on channel=0 note=66 velocity=76 time=0.0125
note_on channel=0 note=64 velocity=72 time=0.008333333333333333
note_on channel=0 note=64 velocity=0 time=0.16666666666666666
note_on channel=0 note=66 velocity=0 time=0.0125
note_on channel=0 note=73 velocity=0 time=0.09583333333333333
note_on channel=0 note=80 velocity=0 time=0.0125
note_on channel=0 note=70 velocity=0 time=0
note_on channel=0 note=61 velocity=83 time=0.07083333333333333
note_on channel=0 note=61 velocity=0 time=0.03333333333333333
note_on channel=0 note=54 velocity=54 time=0.32916666666666666
note_on channel=0 note=54 velocity=0 time=0.08333333333333333
note_on channel=0 note=64 velocity=53 time=0.25416666666666665
note_on channel=0 note=67 velocity=57 time=0.016666666666666666
note_on channel=0 note=70 velocity=62 time=0.1875
note_on channel=0 note=64 velocity=0 time=0.125
note_on channel=0 note=58 velocity=60 time=0
note_on channel=0 note=60 velocity=49 time=0.0125

note_on channel=0 note=63 velocity=0 time=0.10416666666666667
note_on channel=0 note=58 velocity=82 time=0.029166666666666667
note_on channel=0 note=61 velocity=83 time=0
control_change channel=0 control=64 value=0 time=0
note_on channel=0 note=70 velocity=77 time=0.016666666666666666
note_on channel=0 note=73 velocity=82 time=0.004166666666666667
control_change channel=0 control=64 value=127 time=0.075
note_on channel=0 note=70 velocity=0 time=0.041666666666666664
note_on channel=0 note=73 velocity=0 time=0.05
note_on channel=0 note=65 velocity=76 time=0.0125
note_on channel=0 note=58 velocity=0 time=0.0125
note_on channel=0 note=61 velocity=0 time=0.0125
note_on channel=0 note=65 velocity=0 time=0.0375
note_on channel=0 note=63 velocity=83 time=0.06666666666666667
note_on channel=0 note=63 velocity=0 time=0.058333333333333334
note_on channel=0 note=63 velocity=83 time=0.04583333333333333
note_on channel=0 note=63 velocity=0 time=0.06666666666666667
note_on channel=0 note=63 velocity=

note_on channel=0 note=57 velocity=0 time=0.0125
note_on channel=0 note=42 velocity=0 time=0.008333333333333333
control_change channel=0 control=67 value=104 time=0.11666666666666667
control_change channel=0 control=67 value=88 time=0.008333333333333333
control_change channel=0 control=67 value=72 time=0.008333333333333333
control_change channel=0 control=67 value=56 time=0.0125
control_change channel=0 control=67 value=40 time=0
control_change channel=0 control=67 value=24 time=0.020833333333333332
control_change channel=0 control=67 value=0 time=0.004166666666666667
note_on channel=0 note=49 velocity=59 time=0.0875
note_on channel=0 note=57 velocity=54 time=0.3125
note_on channel=0 note=58 velocity=67 time=0.0625
note_on channel=0 note=61 velocity=66 time=0.004166666666666667
note_on channel=0 note=42 velocity=63 time=0.020833333333333332
note_on channel=0 note=61 velocity=0 time=0.0125
note_on channel=0 note=58 velocity=0 time=0.016666666666666666
note_on channel=0 note=57 velocity=

In [ ]:
for msg in midy:
    print(msg)

<meta message track_name name='untitled' time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message key_signature key='C' time=0>
<meta message set_tempo tempo=500000 time=0>
<meta message midi_port port=0 time=0>
<meta message track_name name='Piano' time=0>
control_change channel=0 control=0 value=0 time=0
control_change channel=0 control=32 value=0 time=0
program_change channel=0 program=0 time=0
control_change channel=0 control=64 value=127 time=2.0708333333333333
control_change channel=0 control=64 value=0 time=0.10833333333333334
control_change channel=0 control=64 value=127 time=0.2833333333333333
control_change channel=0 control=67 value=127 time=0.0375
note_on channel=0 note=69 velocity=54 time=0.13333333333333333
note_on channel=0 note=46 velocity=73 time=0.25416666666666665
note_on channel=0 note=89 velocity=52 time=0.5125
note_on channel=0 note=57 velocity=60 time=0.12083333333333333
note_on channel

In [ ]:
midy.ticks_per_beat

120

In [ ]:
int(X.shape[0]/batch_size)

407